In [1]:
import os
import pandas as pd
import urllib
import ffmpeg
import moviepy.editor as mp
import cv2
import numpy as np
import io
import PIL
import random
import math
import shutil
import pickle
import base64
from tqdm.notebook import tqdm
from google.oauth2 import service_account
from google.cloud import vision
from PIL import Image

os.chdir("/nfs/sloanlab003/projects/arc_proj/story/data/sb_ad_links")
LINK_DIR = "/nfs/sloanlab003/projects/arc_proj/story/data/sb_ad_links"
PARENT_DIR = "/nfs/sloanlab003/projects/arc_proj/story/data/sb_ad_vids"

print('done')

done


In [2]:
from google.cloud import videointelligence_v1 as videointelligence
from google.cloud import vision

os.chdir("/nfs/sloanlab003/projects/arc_proj/story/code/nw/google-cloud-sdk")
GOOGLE_APPLICATION_CREDENTIALS = "/nfs/sloanlab003/projects/arc_proj/story/code/nw/ad-story-arc-b60c485322b4.json"
print('done')

done


# getting faces


In [3]:
def detect_person(local_file_path):
    """Detects people in a video from a local file."""
    from google.oauth2 import service_account
    from google.cloud import vision
    import io

    credentials = service_account.Credentials.from_service_account_file("/nfs/sloanlab003/projects/arc_proj/story/code/nw/ad-story-arc-b60c485322b4.json")
    
    client = videointelligence.VideoIntelligenceServiceClient(credentials=credentials)

    with io.open(local_file_path, "rb") as f:
        input_content = f.read()

    # Configure the request
    config = videointelligence.FaceDetectionConfig(
        include_bounding_boxes=True, include_attributes=True
    )
    context = videointelligence.VideoContext(face_detection_config=config)

    # Start the asynchronous request
    operation = client.annotate_video(
        request={
            "features": [videointelligence.Feature.FACE_DETECTION],
            "input_content": input_content,
            "video_context": context,
        }
    )

    print("\nProcessing video for face detection annotations.")
    result = operation.result(timeout=300)

    print("\nFinished processing.\n")

    # Retrieve the first result, because a single video was processed.
    annotation_result = result.annotation_results[0]
    
    return_list = []

    for annotation in annotation_result.face_detection_annotations:
        face_info = []
        
        # print("Face detected:")
        for track in annotation.tracks:

            timestamped_object = track.timestamped_objects[0]
            box = timestamped_object.normalized_bounding_box
            
            
            face_info.append({"box": (box.left, box.top, box.right, box.bottom)})
            face_info.append({"start_time": track.segment.start_time_offset.seconds + track.segment.start_time_offset.microseconds / 1e6})
            face_info.append({"end_time": track.segment.end_time_offset.seconds + track.segment.end_time_offset.microseconds / 1e6})



            return_list.append(face_info)
    # print('here', return_list)
    return return_list

In [4]:
def crop_from_video(ID, bounding_box, timestamp):
    os.chdir(f'{PARENT_DIR}/{ID}')
    filename = ID + '.mp4'
    
    vidcap = cv2.VideoCapture(filename)
    fps = vidcap.get(cv2.CAP_PROP_FPS)
    success,image = vidcap.read()
    count = 0
    success = True

    while success:
        success,frame = vidcap.read()

        count+=1
        if count/fps >= timestamp:
            # print('frame #', count, 'time', count/fps)
            imageRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            im = Image.fromarray(imageRGB)
            roi = crop_normalized(bounding_box, im)
            
            vidcap.release()
            cv2.destroyAllWindows()
            return roi
            break
            
    vidcap.release()
    cv2.destroyAllWindows()


In [5]:
def crop_normalized(bounding_box, image):
    left, top, right, bottom = bounding_box[0], bounding_box[1], bounding_box[2], bounding_box[3]
    number_of_rows, number_of_columns = image.height, image.width
    
    top_unormalized = round(top * number_of_rows)
    left_unormalized = round(left * number_of_columns)
    bottom_unormalized = round(bottom * number_of_rows)
    right_unormalized = round(right * number_of_columns)
    
    return image.crop((left_unormalized, top_unormalized, right_unormalized, bottom_unormalized))

In [8]:
# run videos through video intelligence API
os.chdir(LINK_DIR)
df = pd.read_csv("nw_2014_included.csv")
df_working = pd.read_csv("urop-relabeled.csv")

num_faces_save = 10
df2 = df.copy()
list_face = list(df["faces detected"])

for i in range(3578, df.shape[0]):
    
    # if pd.isnull(df_working["working_links_to_vid"][i]): 
    #     continue
    # if df_working["working_links_to_vid"][i] == 'no vid found':
    #     continue
    if df['needs_processing'][i] == 0:
        continue
    
    ID = df["id"][i]
    os.chdir(f'{PARENT_DIR}/{ID}')
    
    # try:
    result = detect_person(ID+".mp4")
    segment_length = {}
    
    try:
        os.mkdir(f"{PARENT_DIR}/{ID}/google_video_face_samples")
    except:
        os.chdir(f"{PARENT_DIR}/{ID}/google_video_face_samples")

    for j, face in enumerate(result[::-1]):
        bb = face[0]['box']
        start_time = face[1]['start_time']
        end_time = face[2]['end_time']
        segment_length[end_time - start_time] = (bb, start_time)

    num_faces = len(segment_length.keys())
    top_faces = sorted(segment_length.keys(), reverse=True)[:num_faces_save]

    # df2['faces detected'][i] = num_faces
    list_face[i] = num_faces

    # if i % 500 == 0 and i != 0:
    #     os.chdir(LINK_DIR)
    #     df2.to_csv(f'temp_{i-500}-{i}.csv', index=False)


    for j, segment in enumerate(top_faces):
        (bb, start_time) = segment_length[segment]
        roi = crop_from_video(ID, bb, start_time)
        roi.save(f"{PARENT_DIR}/{ID}/google_video_face_samples/{str(j).zfill(3)}_{str(round(start_time, 2)).zfill(5)}.jpg")

    print('done with', ID)
    # except:
    #     didnt_work.append(ID)
    #     print(ID, "didn't work")
        
df2["faces detected"] = list_face
os.chdir(LINK_DIR)
df2.to_csv('nw_2014_included_v2.csv', index=False)
print('done!')


Processing video for face detection annotations.

Finished processing.

done with sb_2014_3583

Processing video for face detection annotations.

Finished processing.

done with sb_2014_3584

Processing video for face detection annotations.

Finished processing.

done with sb_2014_3585

Processing video for face detection annotations.

Finished processing.

done with sb_2014_3586

Processing video for face detection annotations.

Finished processing.

done with sb_2014_3587

Processing video for face detection annotations.

Finished processing.

done with sb_2014_3588

Processing video for face detection annotations.

Finished processing.

done with sb_2014_3589

Processing video for face detection annotations.

Finished processing.

done with sb_2014_3590

Processing video for face detection annotations.

Finished processing.

done with sb_2014_3591

Processing video for face detection annotations.

Finished processing.

done with sb_2014_3592

Processing video for face detection ann

In [ ]:
df = pd.read_csv("nw_jan_fixed_visual.csv")
df_working = pd.read_csv("urop-relabeled.csv")